In [ ]:
# setting up jupyter
%matplotlib inline
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 300
mpl.rcParams['savefig.dpi'] = 300

# Example 1: successive differentiation of hESCs to hepatocytes
In this experiment human embryonic stem cells were treated with a selection of stimuli to differentiate into hepatocytes/ liver tissue. Within the a period of 3 weeks, 10 samples were taken and sequenced on different days. This example goes into samples and targets initiation and basic plotting.

## Initializing samples and targets
To <b>initiate the samples instance</b>, we pass a TSV expression file and gene list data. Since there are no replicates in this data, we cannot identify significant differential genes with DESeq2 (of which DPre accepts the output). Instead, in this analysis, differential genes were defined as those with a differential Z-score >2; <-2. Since this is not to uncommon, DPre implements the function <i>add_diff_genes_from_z()</i> to add differential genes based on Z-scores.
Additionally, we pass the name of the control to the <i>ctrl</i> argument to later plot differential effects. We also set a <i>name</i> that will appear in logs and plot annotations.

In [ ]:
import DPre
s = DPre.samples(expression = 'hsliver_expression.tsv',     # expression input data
                 diff_genes = None,    # differential gene are loaded in the next step
                 ctrl = 'Day00',     # name of the control in the data
                 name = 'in vitro hepatic differentiated hESCs'    # name of the samples instance
)
DPre.add_diff_genes_from_z(s)


The <b>targets initiation</b> is easy because we want to compare against the pre-packaged human reference data. This can simply be realized by calling the <i>preset_targets()</i> function with 'human'.

In [ ]:
from DPre import preset_targets
t = preset_targets('human')     # initiate one of the pre-packaged targets, here the human reference

Because we did not set the <i>preset_colors</i> argument to <i>None</i>, targets default colors were set according to the respective domain. In addition, a color legend is created to reference the mapping of domains and colors. 

The <b>command line interface</b> combines targets and samples initiation with producing the plot. Unlike the Python interface,
the command line interface cannot infer the differential genes from Z-scores. Here, we used the <i>samples._diff</i> DataFrame to read out the differential genes and wrote them into two directories.<br> 
Without the plot defined, the command is invalid. But for demonstration purposes the equivalent would be:

In [ ]:
!python ../../dpre.py --preset_targets "human" --samples_expression "hsliver_expression.tsv" --samples_diff_genes "./up_genelists" --samples_diff_genes "./down_genelists" --control "Day00"  --samples_name "in vitro hepatic differentiated hESCs"

or with abbriviated arguments:

In [ ]:
!python ../../dpre.py -pt "human" -se "hsliver_expression.tsv" -sd "./up_genelists" -sd "./down_genelists" -c "Day00"  -sn "in vitro hepatic differentiated hESCs"

Now that both the targets and samples are defined, we can <b>compute and plot the transcriptional similarity</b> between the two.

## Plotting  target transcriptional similarity

First, we want to get an initial overview of the data. To produce this overview, the <b>target similarity heatmap</b> is called from the targets instance. Here we first want to check the <b>'intersect' metric</b>. Before that, we will change the names to something more readable. Note: in the following plot calls we pass the argument plt_show = True. This is simply for displaying the plot in jupyter and will default to False.

In [ ]:
s.names = ['Day 0', 'Day 1', 'Day 2', 'Day 3', 'Day 5', 'Day 7', 'Day 9', 'Day 11', 'Day 13', 'Day 21']

In [ ]:
hm = t.target_similarity_heatmap(samples = s,     # the samples to check similarity for
                                 metric = 'intersect',     # the metric to use for computing similarity
                                 filename = None,     # don't save this initial plot
                                 plt_show = True      # show plot in jupyter
                                )

The log shows normal proportions of detected marker genes in the samples. No targets are dropped for the analysis.
As we can see, the default parameters of the target similarity heatmap are set to show the target labels. The plot is created with the space necessary to cleanly display all target labels. Therefore, when comparing against a large reference the plot widely extends in x-direction. Although this can be useful for reading out the similarities of all targets by zooming in, we cannot present it in this size. To solve this, we pass <i><b>heatmap_width</b></i>, <b><i>targetlabels_space</i></b>, and <b><i>specifc_target_labels</i></b> to only print the labels we care about.

In [ ]:
hm = t.target_similarity_heatmap(samples = s,
                                 metric = 'intersect',    
                                 heatmap_width = .14, 
                                 specific_target_labels = ['Hepatocyte', 'Liver adult', 'Liver fetal'],       # only print a specifc target label instead of all
                                 targetlabels_space = .1,     # size in inches under the heatmap
#                                  filename = None,
                                 plt_show = True
                                )

To further reduce the plot to the essentials, we pass <b><i>hide_distance_bar</i></b> (number of marker genes), <b><i>samplelabels_space</i></b> and set <b><i>title</i></b> to <i>None</i>. Also, we pass <b><i>cluster_targets</i></b> and <b><i>hide_targets_dendrogram</i></b> to emphasize peak values. We can also save the plot with the default filename now by not setting <b>filename</b> to None.

In [ ]:
hm = t.target_similarity_heatmap(samples = s, 
                                 metric = 'intersect',  
                                 heatmap_width = .14,
                                 specific_target_labels = ['Hepatocyte'],
                                 targetlabels_space = .1,
                                 hide_distance_bar = True,      # hide the distance bar, for intersect n marker genes
                                 cluster_targets = True,          # emphasize peak samples and targets
                                 hide_targets_dendrogram = True,  # don't show the dendrogram to save space
                                 samplelabels_space = .3,       # save some space on the left
                                 title = None,                  # remove the title to increase clarity
#                                  filename = None,             # when not passed the default filename is used
                                 plt_show=True
                                )

The same output can be achieved by calling the following <b>command</b>:

In [ ]:
!python ../../dpre.py -pt "human" -se "hsliver_expression.tsv" -sd "./up_genelists" -sd "./down_genelists" -c "Day00"  -sn "in vitro hepatic differentiated hESCs" target_sim -m "intersect" -hw 0.14 -st "Hepatocyte" -ta 0.4 -hd -ct -htd -sa 0.3 -t "False"

## Plotting single gene transcriptional similarity

Next we want to investigate the exact transcriptional similarity with the hepatocyte target for two groups of <b>genes</b> for which we will use the <b>'euclid'</b> metric. First, known hepatocyte marker genes and next the genes driving the similarity increase most dominantly. This functionality is provided by the <i>gene_similarity_heatmap()</i> function. The function will create a single gene similarity plot for every element in the targets instance. Since we are only interested in Hepatocyte similarity, we can <b>slice the elements</b> in targets like so:

In [ ]:
hep = t.slice_elements(['Hepatocyte'], name='Hepatocyte')

In [ ]:
hm = hep.gene_similarity_heatmap(samples = s,              # we don't need to pass metric here, 'euclid' is the default
                                 filename = None,
                                 plt_show = True)

By default, this function will plot the 45 most variant genes, we are however interested in the <b>similarity of a specifc gene list</b>: a collection of hepatoblast- and hepatocyte markers. We define these gene lists and generate a color legend for the two groups. Then we make a dictionary that maps the genes to the respective color. DPre's predefined <b>colors</b> can be accessed from <i>config.colors</i>:

In [ ]:
from DPre import plot_color_legend
from DPre import config
hep_bl = ['PITX2', 'KRT19', 'AFP', 'DAB2', 'FOXP4', 'SMAD3', 'FOXA1', 'HES4', 'HNF1B']
hep_cy = ['ALB', 'MXD1', 'TTR']
plot_color_legend(['Hepatoblast', 'Hepatocytes'], 
                  [config.colors[11], config.colors[12]],
                  filename='hepato_markers_legend.png',
                  ncolumns = 1,
                 )
# set up the colors for the colorbar
colorbar_cols = {**dict.fromkeys(hep_bl, config.colors[11]), 
                 **dict.fromkeys(hep_cy, config.colors[12])}
colorbar_cols

To check the similarity of a custom gene list, this list is passed to <i><b>custom_target_genelist</b></i>. This argument is used when we don't care about the marker genes in the reference. Instead, the passed list of genes is used for all targets. Since we will plot another gene similarity heatmap, we want to control the <i><b>heatmap_range</b></i> instead of inferring it from the data. Due to space constrains, the <b>summary plot</b> on the right is not plotted. We also pass the <b>pivot</b> argument to rotate the plot 90 degrees. This way, we can share the sample labels with the previous target_similarity heatmap. Apart from these arguments, the other inputs overlap with the previous <i>target_similarity_heatmap()</i> function:

In [ ]:
hm = hep.gene_similarity_heatmap(samples = s,    # the sampels to explore similarity for
                                 custom_target_genelist = hep_bl+hep_cy,    # the list of genes to be visualized
                                 heatmap_range = [-6, 6],     # range heatmap values
                                 heatmap_width = .8, 
                                 heatmap_height = .9, 
                                 genelabels_space = .5, 
                                 samplelabels_space = .1, 
                                 title = None,
                                 hide_distance_bar = True, 
                                 show_genes_colorbar = colorbar_cols,    # the colorbar dictionary created above
                                 pivot = True,          # rotate the plot
                                 hide_samplelabels = True,         # in the final plot, shared with the previous plot
                                 hide_sum_plot = True,     # do not plot the summerizing plot on the right
                                 filename = 'hsliver_marker_genes.png',
                                 plt_show = True,
                                )


The <b>command line interface</b> doesn't support custom colors. Apart from this the plot can be reproduced like so:

In [ ]:
!python ../../dpre.py -pt "human" -ts "Hepatocyte" -se "hsliver_expression.tsv" -sd "./up_genelists" -sd "./down_genelists" -c "Day00"  -sn "in vitro hepatic differentiated hESCs" gene_sim -cu "PITX2" -cu "KRT19" -cu "AFP" -cu "DAB2" -cu "FOXP4" -cu "SMAD3" -cu "FOXA1" -cu "HES4" -cu "HNF1B" -cu "ALB" -cu "MXD1" -cu "TTR" -hr -6 -hr 6 -pi -hw 0.8 -hh 0.9 -ge 0.3 -sa 0.1 -ge .5  -t "false" -hd -hsa -hs -f "hsliver_marker_genes.png"

Second, we want to find the 10 most similarity-increasing genes for the hepatocyte fate. Instead of passing <i>custom_target_genelist</i>, we change <b><i>display_genes</i></b> and <b><i>gene_number</i></b>. We can also use this example to remove the white space on the top of the plot. Advanced size adjustments like this are realized by passing the size constants defined in <i>config</i> (see documentation for details), in this case <i>HM_TOP</i>. This functionality is not available through the command line.

In [ ]:
hm = hep.gene_similarity_heatmap(samples = s, 
                                 display_genes = 'increasing',      # the genes with peak positve similarity values
                                 gene_number = 10,        # the number of genes to extract and display
                                 heatmap_range = [-6, 6],
                                 plt_show=True,
                                 pivot = True,
                                 heatmap_width = .65, 
                                 heatmap_height = .9, 
                                 genelabels_space = .5, 
                                 samplelabels_space = .4, 
                                 title = 'Most similarity\nincreasing genes',
                                 hide_distance_bar = True,
                                 hide_samplelabels = True,     
                                 hide_sum_plot = True,     
                                 HM_TOP = .7,                      # the space in inches above the heatmap
                                 filename = 'hsliver_increasing_genes.svg',
                                )

which is equivalent (without adjusting the sum plot size) to the <b>command</b>:

In [ ]:
!python ../../dpre.py -pt "human" -ts "Hepatocyte" -se "hsliver_expression.tsv" -sd "./up_genelists" -sd "./down_genelists" -c "Day00"  -sn "in vitro hepatic differentiated hESCs" gene_sim -gn 10 -di "increasing" -hr -6 -hr 6 -pi -hw 0.8 -hh 0.9 -ge 0.3 -sa 0.1 -ge .5  -t "Most similarity\nincreasing genes" -hd -hs -hsa -hs -f "hsliver_marker_genes.png"

Done. We now have an idea about the genes behind the differentiation bias towards hepatocytes. This is how Figure 1C was generated with DPre.